<a href="https://colab.research.google.com/github/woo-ara/MySoloLife/blob/master/MNIST1_drop%2Cbatch_x.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MNIST 데이터를 사용하여 심층 신경망을 구현하고, 과적합을 방지하기 위한 dropout과 batch normalization을 적용하여 결과를 분석하여 파이썬 코드와 실행결과를 정리한 보고서를 학번-이름.pdf로 변환하여 기한 내에 이클래스에 제출하세요.
 

1. 데이터는 MNIST를 사용합니다.

2. 심층신경망의 입력층, 은닉층 3개(첫번째, 두번째, 세번째 은닉층 노드 개수 각각 256개, 256개, 256개), 출력층으로 되어 있는 모델을 생성하고 그밖의 사항들은 자유롭게 지정하면 됩니다.

3. 최적화 방법은 Gradient Descent Optimization 을 사용합니다.

4. 과적합 방지를 위해 droupout의 수치를 변경하면서 성능을 분석합니다.

5. 과적합 방지를 위해 batch normalization을 수행하여 성능을 분석합니다.

In [ ]:
#python 버전 확인
!python --version

Python 3.7.12


In [ ]:
#tensorflow 버전 확인
import tensorflow as tf
print(tf.__version__)

1.15.0


In [ ]:
# 2.x 버전일 경우 현재 tenserflow 제거
!pip uninstall tensorflow

# 1.15 버전으로 tensorflow 설치
!pip install tensorflow==1.15

Found existing installation: tensorflow 1.15.0
Uninstalling tensorflow-1.15.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-1.15.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
    /usr/local/lib/python3.7/dist-packages/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.15.0
  Using cached tensorflow-1.15.0-cp37-cp37m-manylinux2010_x86_64.whl (412.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.6 requires tensorflow>=2.0.0, but you have tensorflow 1.15.0 which is incompatible.


In [ ]:
#TensorFlow 및 MNIST 데이터셋 임포트
import tensorflow as tf

#데이터를 내려받은 후 레이블을 one-hot encoding 방식으로 읽어들임
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [ ]:
#신경망 모델 구성:입력, 출력 정의
#X와 Y 텐서의 첫번째 차원:None
#한 번에 학습시킬 MNIST 이미지의 개수 지정:배치 크기 지정
X = tf.placeholder(tf.float32, [None, 784]) #영상의 크기가 28x28(=784)
Y = tf.placeholder(tf.float32, [None, 10])

In [ ]:
#3개의 은닉층이 다음처럼 구성된 신경망 생성
#784(입력, 특징 개수) -> 256(첫 번째 은닉층 뉴런 개수) -> 256(두 번째 은닉층 뉴런 개수) -> 256(세 번째 은닉층 뉴런 개수)-> 10(결괏값 0~9 분류 개수)
W1 = tf.Variable(tf.random_normal([784,256],stddev=0.01))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_normal([256,256], stddev=0.01))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_normal([256, 256], stddev=0.01))
L3 = tf.nn.relu(tf.matmul(L2, W3))

W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

In [ ]:
#손실 함수와 최적화 함수 적용
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2
                      (logits=model, labels=Y))

#최적화 방법은 Gradient Descent Optimization
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)
#optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [ ]:
#결과 확인
#신경망 모델 초기화 및 세션 시작
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
  total_cost = 0

  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)

    _, cost_val = sess.run([optimizer, cost], 
                           feed_dict={X: batch_xs, Y: batch_ys})
    total_cost += cost_val
  
  print('Epoch:', '%04d' % (epoch + 1), 'Avg. cost = ', '{:.3f}'.format
        (total_cost / total_batch))

print('최적화 완료!')
                                         

Epoch: 0001 Avg. cost =  2.303
Epoch: 0002 Avg. cost =  2.303
Epoch: 0003 Avg. cost =  2.302
Epoch: 0004 Avg. cost =  2.302
Epoch: 0005 Avg. cost =  2.302
Epoch: 0006 Avg. cost =  2.302
Epoch: 0007 Avg. cost =  2.302
Epoch: 0008 Avg. cost =  2.302
Epoch: 0009 Avg. cost =  2.302
Epoch: 0010 Avg. cost =  2.302
Epoch: 0011 Avg. cost =  2.302
Epoch: 0012 Avg. cost =  2.301
Epoch: 0013 Avg. cost =  2.301
Epoch: 0014 Avg. cost =  2.300
Epoch: 0015 Avg. cost =  2.297
최적화 완료!
